# CS246 - Colab 1
## Word Count in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 29 kB/s 
     |████████████████████████████████| 198 kB 59.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=38cff6c69972f571fbd0988095e7b0c995bd7fcebedc7914ade47e2d16aab991
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you successfully run the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter, we want to count the total number of (non-unique) words that start with a specific letter.

In your implementation, **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all words that **start** with a non-alphabetic character. You should output word counts for the **entire document**, inclusive of the title, author, and the main texts. If you encounter words broken as a result of new lines, e.g. "pro-ject" where the segment after the dash sign is on a new line, no special processing is needed and you can safely consider it as two words.

Your outputs will be graded on a range -- if your differences from the ground-truths are within an error threshold of 5, you'll be considered correct.

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
# getOrCreate() method returns an already existing SparkSession;
# if not exists, it creates a new SparkSession.
spark = SparkSession.builder.getOrCreate()

In [13]:
# Create RDD from external Data source
rdd = spark.sparkContext.textFile("/content/pg100.txt")
# it splits each record by space in an RDD
rdd = rdd.flatMap(lambda x: x.split(" "))
# keep all words only the first letter and convert to lowercase, 
# add a new column with value 1 for each processed word,
rdd = rdd.map(lambda x: (x[0].lower(),1) if x and x.isalpha() else (None, 1))
# it reduces the word string by applying the sum function on value. 
# The result of our RDD contains unique words and their count
rdd = rdd.reduceByKey(lambda a,b: a+b)
rdd = rdd.map(lambda x: (x[1],x[0])).sortByKey(False)
# remove None
rdd = rdd.filter(lambda x : x[1] != None)
#Print rdd result to console
print(rdd.collect())

[(105860, 't'), (72155, 'a'), (52986, 'i'), (46934, 'w'), (44741, 's'), (44569, 'h'), (39937, 'm'), (36408, 'o'), (34065, 'b'), (25624, 'f'), (20646, 'y'), (19618, 'c'), (19492, 'd'), (19183, 'n'), (17256, 'l'), (15838, 'p'), (13682, 'g'), (12666, 'e'), (7973, 'r'), (7008, 'u'), (5861, 'k'), (3361, 'v'), (1474, 'j'), (1299, 'q'), (32, 'z'), (1, 'x')]


Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!